In [1]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

In [2]:
# Fixed parameters
filename = "Tempn05Un3Vn33sid_full.csv"
file = open(filename,"w+")
K = 0.0
U = -3.0
V = -3.3
Temp = 0.001  # Temperature needs to be set up here
nval = 0.5
Lsmall = 12
Llarge = 40
# Fixed functions
def eps(kx,ky):
    f = -2.0*(np.cos(kx) + np.cos(ky))
    return f

def sk(kx, ky):
    f = 1/2*(np.cos(kx) + np.cos(ky))
    return f

def dk(kx, ky):
    f = 1/2*(np.cos(kx) - np.cos(ky))
    return f

def deltaSing(kx,ky,d0,ds,dd):
    sing = (d0 + ds*sk(kx,ky) + (1.0j)*dd*dk(kx,ky))
    return sing

def energy(mu,kx,ky,d0,ds,dd):
    f = np.sqrt((eps(kx, ky) - mu)**2 + np.abs(deltaSing(kx,ky,d0,ds,dd))**2)
    return f

def fermi(T,mu,kx,ky,d0,ds,dd):
    f = 1.0/(np.exp(energy(mu,kx,ky,d0,ds,dd)/T) + 1.0)
    return f

def g(T,mu,kx,ky,d0,ds,dd):
    f = (1.0 - 2.0*fermi(T,mu,kx,ky,d0,ds,dd))/(2.0*energy(mu,kx,ky,d0,ds,dd)+0.0000001)
    return f

In [3]:
Tempend = 1.0
while V<-3.25:
    U = -3.0
    while Temp<Tempend:
        print('nval =', nval,' Temp =', Temp, ' U = ',U)
        L = Lsmall
        NSites = L**2
        delK = 2.0*np.pi/np.sqrt(NSites)
        kvals = [-np.pi + delK + i*delK for i in range(L)]

        def n(mu,T,d0,ds,dd):
            sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],d0,ds,dd) for i in range(len(kvals)) for j in range(len(kvals)))
            f = 1.0-sump
            return f

        sktab = np.zeros((len(kvals),len(kvals)))
        dktab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                sktab[i,j] = sk(kvals[i], kvals[j])
                dktab[i,j] = dk(kvals[i], kvals[j])

        def numeqn(mu,d0,ds,dd):
            return nval - n(mu,Temp,d0,ds,dd)

        def FreeE(p):
            Del0, Dels, Deld = p 

            mu = optimize.root(numeqn, -1.5, args=(Del0,Dels,Deld), method='broyden1', options={'ftol':0.03}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delSingp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            i00 = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iss = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            idd = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Del0,Dels,Deld)
                    delSingp[i,j] = deltaSing(kvals[i],kvals[j],Del0,Dels,Deld)
                    del2p[i,j] = (np.abs(delSingp[i,j])**2)*gp[i,j]
                    i00[i,j] = delSingp[i,j]*gp[i,j]
                    iss[i,j] = sktab[i,j]*i00[i,j]
                    idd[i,j] = dktab[i,j]*i00[i,j]

            I00 = np.abs(1.0/(NSites)*np.sum(i00))**2
            Iss = np.abs(1.0/(NSites)*np.sum(iss))**2
            Idd = np.abs(1.0/(NSites)*np.sum(idd))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            Vterm = U*I00 + 4.0*V*Iss +4.0*V*Idd

            return np.real(enerterm + logterm + mu*nval + Vterm)

        def FreeEs(p):
            Del0, Dels = p 
            Deld = 0.0
            return FreeE((Del0,Dels,Deld))

        def FreeEd(p):
            Deld = p 
            Del0 = 0.0
            Dels = 0.0
            return FreeE((Del0,Dels,Deld))

        freeEval = 10.0;
        for i in range(11):
            r1 = 0.0
            r2 = 0.0
            r3 = np.random.uniform(0.33,5.0)
            f = FreeE((r1,r2,r3))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r3])
                print(globmin,f)

        #globmind=optimize.minimize(FreeEd, globmin, callback=None, options={'gtol': 2e-3, 'disp': True}).x 
        globmind1=np.array([0.06,0.06,globmin[0]])
        globmind1=optimize.minimize(FreeE, globmind1, callback=None, options={'gtol': 4e-3, 'disp': True}).x
        freeEdval1=FreeE(globmind1)

        freeEval = 10.0;
        for i in range(67):
            r1 = np.random.uniform(0.16,2.0)
            r2 = np.random.uniform(-4.0,4.0)
            r3 = 0.0
            f = FreeE((r1,r2,r3))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r1,r2])
                print(globmin,f)

        #globmins=optimize.minimize(FreeEs, globmin, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        globmins1=np.array([globmin[0],globmin[1],0.04])
        globmins1=optimize.minimize(FreeE, globmins1, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        freeEsval1=FreeE(globmins1)


        freelist=np.array([freeEdval1,freeEsval1])
        globminlist=np.array(np.array([globmind1,globmins1]))
        result = np.where(freelist == np.amin(freelist))
        globmin = globminlist[result[0]][0]
        print(globmin, " ", np.amin(freelist))

        # Fixed parameters for larger lattice
        L = Llarge
        NSites = L**2
        delK = 2.0*np.pi/np.sqrt(NSites)
        kvals = [-np.pi + delK + i*delK for i in range(L)]

        def n(mu,T,d0,ds,dd):
            sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],d0,ds,dd) for i in range(len(kvals)) for j in range(len(kvals)))
            f = 1.0-sump
            return f

        sktab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                sktab[i,j] = sk(kvals[i], kvals[j])

        dktab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                dktab[i,j] = dk(kvals[i], kvals[j])

        def FreeE(p):
            Del0, Dels, Deld = p 

            mu = optimize.root(numeqn, -1.5, args=(Del0,Dels,Deld), method='broyden1', options={'ftol':0.008}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delSingp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            i00 = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iss = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            idd = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Del0,Dels,Deld)
                    delSingp[i,j] = deltaSing(kvals[i],kvals[j],Del0,Dels,Deld)
                    del2p[i,j] = (np.abs(delSingp[i,j])**2)*gp[i,j]
                    i00[i,j] = delSingp[i,j]*gp[i,j]
                    iss[i,j] = sktab[i,j]*i00[i,j]
                    idd[i,j] = dktab[i,j]*i00[i,j]

            I00 = np.abs(1.0/(NSites)*np.sum(i00))**2
            Iss = np.abs(1.0/(NSites)*np.sum(iss))**2
            Idd = np.abs(1.0/(NSites)*np.sum(idd))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            Vterm = U*I00 + 4.0*V*Iss +4.0*V*Idd

            return np.real(enerterm + logterm + mu*nval + Vterm)

        globmin=optimize.minimize(FreeE, globmin, callback=None, options={'gtol': 5e-6, 'disp': True}).x
        F=FreeE(globmin)

        mu = optimize.root(numeqn, -1.5, args=(globmin[0],globmin[1],globmin[2]), method='broyden1', options={'ftol':0.008}).x
        data = (Temp, mu, globmin[0],globmin[1],globmin[2], F)
        print(data)
        file.write("%s,%s,%s,%s,%s,%s\n" % data)
        
        Temp = Temp + 0.01
    
    V = V + 0.1
    
file.close()

nval = 0.5  Temp = 0.001  U =  -3.0
[3.42340617] -1.2802895372450722
[3.37431522] -1.2818537234810783
[1.37854934] -1.341975529437668
[1.27771289] -1.3423225495482312
Optimization terminated successfully.
         Current function value: -1.372702
         Iterations: 7
         Function evaluations: 60
         Gradient evaluations: 12
[ 0.22958467 -1.62774007] -1.2925498238731778
[1.62562688 0.60028967] -1.3132180469845944
[0.19220699 0.77692249] -1.3513755267102743
[0.51941344 0.77033224] -1.369931433577881
Optimization terminated successfully.
         Current function value: -1.371732
         Iterations: 8
         Function evaluations: 45
         Gradient evaluations: 9
[0.57072111 0.9623713  0.89928332]   -1.3727019465779338
Optimization terminated successfully.
         Current function value: -1.373063
         Iterations: 14
         Function evaluations: 80
         Gradient evaluations: 16
(0.001, array(-1.48450867), 0.5858048139890485, 0.9058788767430834, 0.7168386068046

(0.07100000000000001, array(-1.48449991), 0.5858225412529706, 0.9059029066842944, 0.7167887274003345, -1.3730631535379625)
nval = 0.5  Temp = 0.081  U =  -3.0
[4.01594091] -1.2627544175954446
[2.97359479] -1.3036630009035395
[1.11355068] -1.3422126658675726
Optimization terminated successfully.
         Current function value: -1.372711
         Iterations: 8
         Function evaluations: 65
         Gradient evaluations: 13
[1.25733463 2.64451266] -1.3269227411511895
[0.79592349 0.03637272] -1.356051671272475
[0.58121381 0.12954871] -1.358565800781214
[0.65356106 0.98210762] -1.3716326405612298
Optimization terminated successfully.
         Current function value: -1.371676
         Iterations: 1
         Function evaluations: 15
         Gradient evaluations: 3
[0.5689748  0.96922572 0.89259933]   -1.3727106546942096
Optimization terminated successfully.
         Current function value: -1.373063
         Iterations: 15
         Function evaluations: 85
         Gradient evaluations

Optimization terminated successfully.
         Current function value: -1.371813
         Iterations: 9
         Function evaluations: 50
         Gradient evaluations: 10
[0.56048963 0.96304134 0.89396841]   -1.3727759409100584
Optimization terminated successfully.
         Current function value: -1.373132
         Iterations: 14
         Function evaluations: 80
         Gradient evaluations: 16
(0.151, array(-1.48472517), 0.5846966461216835, 0.9046108674541115, 0.7159807908433364, -1.3731315206439847)
nval = 0.5  Temp = 0.161  U =  -3.0
[3.50293191] -1.2877903376006847
[2.00033054] -1.3355694097640474
[0.76977642] -1.3403451954976249
[0.87419185] -1.3420999299833563
[1.11596112] -1.3443845945933774
Optimization terminated successfully.
         Current function value: -1.372939
         Iterations: 8
         Function evaluations: 70
         Gradient evaluations: 14
[ 1.65156346 -0.93651453] -1.2694787143722845
[0.29877209 2.99027747] -1.294251855560246
[0.7996705  2.81654377] -1.

[0.98192926 1.08341205] -1.363794675749643
[0.4479407  0.40077767] -1.364461900608812
Optimization terminated successfully.
         Current function value: -1.373099
         Iterations: 9
         Function evaluations: 50
         Gradient evaluations: 10
[0.58551444 0.92841708 0.8280842 ]   -1.3740960254695762
Optimization terminated successfully.
         Current function value: -1.374296
         Iterations: 17
         Function evaluations: 90
         Gradient evaluations: 18
(0.23100000000000007, array(-1.48666417), 0.5735594003072174, 0.8913421477241519, 0.7014235451321419, -1.3742955932588905)
nval = 0.5  Temp = 0.24100000000000008  U =  -3.0
[0.61770914] -1.345923927968994
[1.03454166] -1.3508565166265878
Optimization terminated successfully.
         Current function value: -1.374423
         Iterations: 9
         Function evaluations: 80
         Gradient evaluations: 16
[ 0.45613424 -3.30851908] -1.196827326586679
[ 1.65671895 -0.86706973] -1.272473475794622
[0.1895841  

[0.88918546 1.14028433] -1.3706613172763766
[0.69821789 0.65161913] -1.3764729122602675
Optimization terminated successfully.
         Current function value: -1.378300
         Iterations: 5
         Function evaluations: 35
         Gradient evaluations: 7
[0.5335055  0.88518364 0.7895494 ]   -1.37893508233735
Optimization terminated successfully.
         Current function value: -1.379130
         Iterations: 15
         Function evaluations: 85
         Gradient evaluations: 17
(0.3110000000000001, array(-1.49165074), 0.542514433202046, 0.8521561060666842, 0.6409813969690907, -1.379129805928883)
nval = 0.5  Temp = 0.3210000000000001  U =  -3.0
[4.37997469] -1.2650595085515692
[0.37644584] -1.3585006887976374
Optimization terminated successfully.
         Current function value: -1.379737
         Iterations: 5
         Function evaluations: 60
         Gradient evaluations: 12
[1.54679735 0.92396229] -1.3256034722019194
[0.92342138 2.10721519] -1.353124926000441
[0.35420267 0.50662

...

Optimization terminated successfully.
         Current function value: -1.476503
         Iterations: 16
         Function evaluations: 95
         Gradient evaluations: 19
(0.6210000000000003, array(-1.61794889), -1.9672741703099923e-05, -2.9097567165871867e-05, -1.8488749271476274e-05, -1.4765030660790268)
nval = 0.5  Temp = 0.6310000000000003  U =  -3.0
[2.03868017] -1.4402221260148664
[1.4402824] -1.4615894923760981
[0.73545886] -1.4770170738035227
Optimization terminated successfully.
         Current function value: -1.481716
         Iterations: 7
         Function evaluations: 45
         Gradient evaluations: 9
[ 1.20218636 -2.60201182] -1.2883922802899133
[1.04271063 1.4289536 ] -1.4243053029844628
[0.72995816 1.04709294] -1.4551051505489516
[ 0.52449804 -0.10768531] -1.467084916151434
[0.53065837 0.13969343] -1.4698601738024883
[ 0.20522095 -0.55434795] -1.4725687099472287
Optimization terminated successfully.
         Current function value: -1.481728
         Iterations: 9

[0.33483175 1.07722054] -1.504267483617167
[ 0.35141321 -0.41744637] -1.5078679067788463
[0.39563879 0.15488985] -1.5124250762915865
[0.28637953 0.70661477] -1.5126878115982803
Optimization terminated successfully.
         Current function value: -1.520324
         Iterations: 8
         Function evaluations: 50
         Gradient evaluations: 10
[-0.00455221 -0.01245964  0.00171772]   -1.520323826892341
Optimization terminated successfully.
         Current function value: -1.520327
         Iterations: 9
         Function evaluations: 55
         Gradient evaluations: 11
(0.7010000000000004, array(-1.66039954), 3.529619236390761e-07, 1.9015740364929703e-05, 1.3108163077534834e-05, -1.5203269901289063)
nval = 0.5  Temp = 0.7110000000000004  U =  -3.0
[2.94360177] -1.4330318000772153
[2.20652462] -1.4699011365804115
[0.43586799] -1.5239296581985255
[0.35512976] -1.5246647009346141
Optimization terminated successfully.
         Current function value: -1.526055
         Iterations: 6
  

Optimization terminated successfully.
         Current function value: -1.568267
         Iterations: 6
         Function evaluations: 40
         Gradient evaluations: 8
[1.91185756 3.01274256] -1.3600203527157708
[0.76226377 3.98406718] -1.3864933770377743
[0.23472671 3.15589986] -1.4416360762323792
[0.17410166 1.24696233] -1.545956165650235
[0.3428789 0.8695368] -1.5534311657975448
[0.45759401 0.55436263] -1.5536708980877938
Optimization terminated successfully.
         Current function value: -1.568282
         Iterations: 7
         Function evaluations: 45
         Gradient evaluations: 9
[-0.00873093 -0.00758539  0.00083136]   -1.5682817584908624
Optimization terminated successfully.
         Current function value: -1.568287
         Iterations: 7
         Function evaluations: 45
         Gradient evaluations: 9
(0.7810000000000005, array(-1.70562534), 1.696004895778215e-05, 8.688004478395467e-06, 2.168194596421557e-05, -1.5682870451830047)
nval = 0.5  Temp = 0.79100000000000

Optimization terminated successfully.
         Current function value: -1.613326
         Iterations: 7
         Function evaluations: 45
         Gradient evaluations: 9
(0.8510000000000005, array(-1.74739379), 2.0462183265426633e-05, -0.00011565743312025308, 4.797335767898926e-07, -1.6133258927660585)
nval = 0.5  Temp = 0.8610000000000005  U =  -3.0
[3.1168665] -1.4994622459884357
[2.87197343] -1.5141861423821588
[0.80362541] -1.610160694553816
Optimization terminated successfully.
         Current function value: -1.619976
         Iterations: 6
         Function evaluations: 40
         Gradient evaluations: 8
[ 0.57614118 -2.28332864] -1.5108343580226522
[ 1.16555602 -0.41694917] -1.5261627497009502
[1.12862556 0.10545577] -1.5366035183928122
[0.45734792 1.87343886] -1.5610089106010279
[ 0.43554464 -0.43617904] -1.6020307725919083
[0.2153121  0.69030312] -1.6105708398356877
[ 0.1812077  -0.16734246] -1.6169224609642923
Optimization terminated successfully.
         Current functio

Optimization terminated successfully.
         Current function value: -1.667969
         Iterations: 6
         Function evaluations: 40
         Gradient evaluations: 8
[-4.74413221e-03 -1.62423468e-03 -1.80450095e-05]   -1.6679690679203483
Optimization terminated successfully.
         Current function value: -1.667971
         Iterations: 6
         Function evaluations: 40
         Gradient evaluations: 8
(0.9310000000000006, array(-1.79753724), 1.0388384050640412e-06, 1.4877713578043388e-06, -1.269255132328859e-07, -1.6679708526945136)
nval = 0.5  Temp = 0.9410000000000006  U =  -3.0
[3.23257503] -1.5391131491467722
[0.44490722] -1.6717382316427563
Optimization terminated successfully.
         Current function value: -1.675020
         Iterations: 6
         Function evaluations: 40
         Gradient evaluations: 8
[ 1.03922128 -1.15927565] -1.5763266053402525
[0.90434602 1.57775639] -1.590288491266084
[0.8090407  1.60765163] -1.598603489542578
[0.37762689 1.77949294] -1.6194646